# MKIV location analysis
Using the MKIV file to find the angle and the distance position on the Polar data

In [1]:
from scipy.io import loadmat
import pandas as pd 
import numpy as np
from datetime import datetime
from datetime import timedelta

In [95]:
def windows_timestamp_to_python_datetime(x):
    """
    This method is used to convert a windows timestamp to a python datetime
    """
    return datetime(1601,1,1) + timedelta(microseconds=x/10)

def creating_a_mkiv_df(mdata, mcolumn = ['time','x','y']):
    """
    This method takes a list that contains the time, distance (x position) and the angle (y position) position
    of the MKIV in the Polar Data and returns a pandas DataFrame 
    """
    mat_dataframe = pd.DataFrame(mdata, columns=mcolumn)
    
    # This function is used to convert all the windows timestamp fields in the mat_dataframe to Python DateTime
    return convert_timestamp_to_python_datatime_index_for_MKIV(mat_dataframe) 

def convert_timestamp_to_python_datatime_index_for_MKIV(df):
    """
    This method takes a Pandas DataFrame with a column name 'time' and converts it to Python DateTime.
    The column 'time' will be removed from the DataFrame and the converted Python DateTime will be placed 
    as the new index for the column x and y  
    """
    # Getting the time column from the data frame 
    time_column = df.time
    
    # Converting the window's timestamp to Python DateTime one by one and appending it to a list 
    python_datetime = []
    
    for x in time_column:
        python_datetime.append(windows_timestamp_to_python_datetime(x)) 
        
    # Creating a new dataframe with the x and y column and making Python datetime as the index 
    x_and_y_df = df[['x','y']] 
    x_and_y_df.index=python_datetime
    
    print(x_and_y_df)
    return x_and_y_df

In [91]:
def finding_min_and_max_for_xy(df):
    """
    This method takes a Pandas DataFrame with a DateTime index and a column name 'x' and 'y'.
    The function will grab the DateTime index's columns that are higher or equal to 2016-12-29 and lower than 2016-12-31 
    and finds the min and max value for the x and y columns. 
    """
    
    # Getting columns and rows where index is higher or equal to 2016-12-29 and lower than 2016-12-31 
    df = df[ ((df.index >= "2016-12-29") & 
              (df.index < "2016-12-31") ) ]
    
    # Getting the max and min value for x and y 
    min_and_max_values = { 'x_min' : df.x.min(), 
                           'x_max' : df.x.max(),
                           'y_min' : df.y.min(),
                           'y_max' : df.y.max()}
    
    return identifying_polar_data_subset_position(min_and_max_values)


def identifying_polar_data_subset_position(min_and_max_values):
    """
    This method takes min and max value of x and y columns and calculates the subset to use for the Polar Data 
    """
    x = {'x_min' : min_and_max_values['x_min'], 
         'x_max' : min_and_max_values['x_max']}
    
    y = {
         'y_min' : min_and_max_values['y_min'], 
         'y_max' : min_and_max_values['y_max']
        }
   
    x_difference = x['x_max'] - x['x_min'] #95.0 
    y_difference = y['y_max'] - y['y_min'] #38.0
    
    x_y_positions = { 'x_position' : {'x_start_position' : x['x_min'] - ((256 - x_difference) / 2),
                                      'x_end_position' : ((256 - x_difference) / 2) +  x['x_max']},
                     
                      'y_position' : {'y_start_position' : y['y_min'] - ((256 - y_difference) / 2),
                                      'y_end_position' : ((256 - y_difference) / 2) +  y['y_max']}
                    }  
    return x_y_positions

In [96]:
mat_fn = r"mat/PulseBinM4Dec.mat"
mat = loadmat(mat_fn)
mdata = mat['PulseBinM4']

x_and_y_df = creating_a_mkiv_df(mdata) 

                                 x      y
2016-11-30 23:59:00.000004  3399.0  325.0
2016-12-01 00:00:59.999996  3403.0  326.0
2016-12-01 00:08:59.999998  3413.0  331.0
2016-12-01 00:11:00.000002  3416.0  331.0
2016-12-01 00:11:00.000002  3416.0  331.0
2016-12-01 00:19:00.000004  3424.0  337.0
2016-12-01 00:20:59.999996  3424.0  339.0
2016-12-01 00:20:59.999996  3424.0  339.0
2016-12-01 00:28:59.999996  3424.0  340.0
2016-12-01 00:31:00.000000  3425.0  341.0
2016-12-01 00:39:00.000002  3426.0  341.0
2016-12-01 00:41:00.000004  3426.0  341.0
2016-12-01 00:41:00.000004  3426.0  341.0
2016-12-01 00:48:59.999996  3428.0  340.0
2016-12-01 00:50:59.999998  3428.0  340.0
2016-12-01 00:50:59.999998  3428.0  340.0
2016-12-01 00:59:00.000000  3429.0  339.0
2016-12-01 01:01:00.000004  3428.0  338.0
2016-12-01 01:09:00.000004  3429.0  337.0
2016-12-01 01:10:59.999996  3428.0  338.0
2016-12-01 01:10:59.999996  3428.0  338.0
2016-12-01 01:18:59.999998  3428.0  336.0
2016-12-01 01:21:00.000002  3427.0

In [93]:
start_and_end_postion = finding_min_and_max_for_xy(x_and_y_df)

In [94]:
start_and_end_postion

{'x_position': {'x_end_position': 3525.5, 'x_start_position': 3269.5},
 'y_position': {'y_end_position': 471.0, 'y_start_position': 215.0}}